### Import packages and read data

In [23]:
import pandas as pd
import gzip
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Digital_Music.json.gz')

# df.to_csv('reviews_Digital_Music_5.csv', index=True, sep=',')
# music_df = pd.read_csv('reviews_Digital_Music_5.csv')

In [46]:
Polarity = pd.read_csv("Polarity.csv")

In [48]:
sum(Polarity['review_polarity'] == 0)/(Polarity['review_polarity'].shape[0])

0.74

### We found that around 75% of polarity score equals to zero, so we considers most reviews as product descriptions. 

--------

In [24]:
df1 = df

In [25]:
df1.shape

(836006, 9)

In [26]:
df1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2EFCYXHNK06IS,5555991584,"Abigail Perkins ""Abby &#34;Reads Too Much&#34...","[4, 5]",The anthemic title track begins &quot;The Memo...,5.0,Enya Experiments And Succeeds,978480000,"01 3, 2001"
1,A1WR23ER5HMAA9,5555991584,AKB,"[1, 1]",Just when I thought Enya couldn't possibly get...,5.0,How to improve upon perfection.,953424000,"03 19, 2000"
2,A2IR4Q0GPAFJKW,5555991584,Alexander,"[0, 0]",Nice CD for easy listening. My husband and I ...,4.0,Good listening.,1393545600,"02 28, 2014"
3,A2V0KUVAB9HSYO,5555991584,Alison Hight,"[0, 1]","I really liked this CD, especially the differe...",4.0,Loved It,966124800,"08 13, 2000"
4,A1J0GL9HCA7ELW,5555991584,"Al the Pal ""Al the Pal""","[3, 3]",Enya's richly chorded style has smitten me onc...,5.0,Another Hauntingly Beautiful Collection of Songs,1007683200,"12 7, 2001"


In [27]:
df1 = df1.drop(columns=['reviewerID', 'reviewerName','reviewText', 'summary', 'unixReviewTime','reviewTime'])

In [28]:
df1.head(20)

,asin,helpful,overall
0,5555991584,"[4, 5]",5.0
1,5555991584,"[1, 1]",5.0
2,5555991584,"[0, 0]",4.0
3,5555991584,"[0, 1]",4.0
4,5555991584,"[3, 3]",5.0
5,5555991584,"[3, 3]",5.0
6,5555991584,"[1, 3]",3.0
7,5555991584,"[0, 0]",5.0
8,5555991584,"[6, 8]",5.0
9,5555991584,"[1, 1]",5.0


### Feature "helpful" means the supporting level of each rating. [4,5] means there are four people in five people considers this rating is helpful

In [29]:
df1['helpful'][0:10]

0    [4, 5]
1    [1, 1]
2    [0, 0]
3    [0, 1]
4    [3, 3]
5    [3, 3]
6    [1, 3]
7    [0, 0]
8    [6, 8]
9    [1, 1]
Name: helpful, dtype: object

In [30]:
df['overall'].unique()

array([5., 4., 3., 1., 2.])

### Convert "helpful" to support rate ---- (helpful) / (helpful+not helpful)

In [31]:
support_level = df1['helpful']
overall_score = df1['overall']

In [32]:
support = []
for i in support_level:
    if i[1] == 0:
        support.append(0)
    else:
        support.append(i[0]/i[1])                 

### For each products, we re-score as formula: score = sum(overall * (1+support_rate)) / sum(1+support_rate)

In [34]:
df1['support_level'] = support
df1['support_level'] = df1['support_level']+1

In [35]:
df1['overall_by_support'] = df1['overall'] * (df1['support_level'])

In [36]:
df1.head()

,asin,helpful,overall,support_level,overall_by_support
0,5555991584,"[4, 5]",5.0,1.8,9.0
1,5555991584,"[1, 1]",5.0,2.0,10.0
2,5555991584,"[0, 0]",4.0,1.0,4.0
3,5555991584,"[0, 1]",4.0,1.0,4.0
4,5555991584,"[3, 3]",5.0,2.0,10.0


### Group by "product ID", and calculate average. Eventually, each product will have one final score.

In [37]:
Overall_Rating = df1.groupby(['asin']).sum()

In [38]:
Overall_Rating.head()

,overall,support_level,overall_by_support
asin,,,
5555991584,603.0,199.792516,933.695912
6308051551,10.0,3.000000,15.000000
7901622466,82.0,23.888889,113.944444
B0000000ZW,91.0,32.522222,141.722222
B00000016T,571.0,194.977831,936.249175


In [39]:
Overall_Rating['Final_Score'] = Overall_Rating['overall_by_support']/Overall_Rating['support_level']

In [40]:
Overall_Rating[0:20]

,overall,support_level,overall_by_support,Final_Score
asin,,,,
5555991584,603.0,199.792516,933.695912,4.673328
6308051551,10.0,3.000000,15.000000,5.000000
7901622466,82.0,23.888889,113.944444,4.769767
B0000000ZW,91.0,32.522222,141.722222,4.357704
B00000016T,571.0,194.977831,936.249175,4.801824
B00000016W,2660.0,921.343630,4155.431823,4.510187
B00000017R,391.0,128.462907,615.620824,4.792207
B0000001BA,72.0,24.709091,118.545455,4.797645
B0000001BO,14.0,3.500000,16.500000,4.714286


### Final Score Looks like

In [41]:
Overall_Rating = Overall_Rating[['Final_Score']]

In [42]:
Overall_Rating.head(20)

,Final_Score
asin,
5555991584,4.673328
6308051551,5.000000
7901622466,4.769767
B0000000ZW,4.357704
B00000016T,4.801824
B00000016W,4.510187
B00000017R,4.792207
B0000001BA,4.797645
B0000001BO,4.714286


### Writing out to csv file for final recommendation system.

In [43]:
Overall_Rating.to_csv("overall_rating.csv", header = True)